                                                    CLASSIFICATION IN MACHINE LEARNING USING
                                                    i)logistic regression(sklearn)
                                                    ii)neural network
                                                    iii)random forest classifier
                                                    

IMPORTING RELEVANT LIBRARIES

In [87]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LogisticRegression




In [88]:
df_preprocessed=pd.read_csv("Absenteeism_preprocessed_data.csv")

In [89]:
df_preprocessed

,Unnamed: 0,reason1,reason2,reason3,reason4,month,week_day,Transportation Expense,Distance to Work,Age,Daily Work Load Average,Body Mass Index,Education,Children,Pets,Absenteeism Time in Hours
0,0,0,0,0,1,7,2,1.005844,0.412816,-0.536062,-0.806331,0.767431,1,2,1,4
1,1,0,0,0,0,7,2,-1.574681,-1.141882,2.130803,-0.806331,1.002633,1,1,0,0
2,2,0,0,0,1,7,3,-0.654143,1.426749,0.248310,-0.806331,1.002633,1,0,0,2
3,3,1,0,0,0,7,4,0.854936,-1.682647,0.405184,-0.806331,-0.643782,1,2,0,4
4,4,0,0,0,1,7,4,1.005844,0.412816,-0.536062,-0.806331,0.767431,1,2,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
695,695,1,0,0,0,5,3,-0.654143,-0.533522,0.562059,-0.853789,-1.114186,0,2,0,8
696,696,1,0,0,0,5,3,0.040034,-0.263140,-1.320435,-0.853789,-0.643782,1,1,2,3
697,697,1,0,0,0,5,4,1.624567,-0.939096,-1.320435,-0.853789,-0.408580,0,0,0,8
698,698,0,0,0,1,5,4,0.190942,-0.939096,-0.692937,-0.853789,-0.408580,0,0,0,2


In [90]:
df_preprocessed['Targets']=np.where(df_preprocessed["Absenteeism Time in Hours"]>np.median(df_preprocessed['Absenteeism Time in Hours']),1,0) #define categorical target

df_preprocessed=df_preprocessed.drop("Absenteeism Time in Hours",axis=1) #drop original target as we are only concerned with the binary outcomes


inputs=df_preprocessed.iloc[:,0:14] 

(df_preprocessed)


,Unnamed: 0,reason1,reason2,reason3,reason4,month,week_day,Transportation Expense,Distance to Work,Age,Daily Work Load Average,Body Mass Index,Education,Children,Pets,Targets
0,0,0,0,0,1,7,2,1.005844,0.412816,-0.536062,-0.806331,0.767431,1,2,1,1
1,1,0,0,0,0,7,2,-1.574681,-1.141882,2.130803,-0.806331,1.002633,1,1,0,0
2,2,0,0,0,1,7,3,-0.654143,1.426749,0.248310,-0.806331,1.002633,1,0,0,0
3,3,1,0,0,0,7,4,0.854936,-1.682647,0.405184,-0.806331,-0.643782,1,2,0,1
4,4,0,0,0,1,7,4,1.005844,0.412816,-0.536062,-0.806331,0.767431,1,2,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
695,695,1,0,0,0,5,3,-0.654143,-0.533522,0.562059,-0.853789,-1.114186,0,2,0,1
696,696,1,0,0,0,5,3,0.040034,-0.263140,-1.320435,-0.853789,-0.643782,1,1,2,0
697,697,1,0,0,0,5,4,1.624567,-0.939096,-1.320435,-0.853789,-0.408580,0,0,0,1
698,698,0,0,0,1,5,4,0.190942,-0.939096,-0.692937,-0.853789,-0.408580,0,0,0,0


In [91]:

targets=df_preprocessed['Targets']
targets=np.array(targets)
np.unique(targets)

array([0, 1])

In [92]:
print(inputs.shape)
print(targets.shape)

(700, 14)
(700,)


Split the datset into training and testing part

In [93]:
x_train,x_test,y_train,y_test=train_test_split(inputs,targets,train_size=0.85,random_state=10)
print(x_test.shape,x_train.shape ,"\n",y_test.shape,y_train.shape)
x_train,x_val,y_train,y_val=train_test_split(x_train,y_train,train_size=0.9,random_state=10)
print(x_val.shape,x_train.shape ,"\n",y_val.shape,y_train.shape)

(105, 14) (595, 14) 
 (105,) (595,)
(60, 14) (535, 14) 
 (60,) (535,)


I. USING LOGISTIC REGRESSION 

In [94]:
reg=LogisticRegression()
reg.fit(x_train,y_train)
print("score using builtin function:",reg.score(x_test,y_test))
print("\nchecking manually")
predicted=reg.predict(x_test)
accuracy_matrix=np.array([int(val) for val in predicted==y_test])
print("\t\t\t\taccuracy_matrix\n",accuracy_matrix)
print("\nscore:",sum(accuracy_matrix)/accuracy_matrix.shape[0])

score using builtin function: 0.7523809523809524

checking manually
				accuracy_matrix
 [0 1 1 0 1 0 1 1 1 1 1 1 0 0 0 1 1 0 1 1 1 1 0 1 1 1 1 0 1 0 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 0 1 1 1 1 1 1 1 1 1 0 0 1 1 1 1 1 1 1 1 0 0
 1 0 0 1 1 1 0 1 1 0 1 1 0 1 1 1 0 0 0 1 1 1 0 1 1 0 1 1 1 1 1]

score: 0.7523809523809524


C:\Users\HP\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [95]:
features=inputs.columns.values
coefficents=reg.coef_[0]

df_reg=pd.DataFrame({"features": features,"coef":coefficents})
df_reg=pd.concat([df_reg,pd.DataFrame({"features":"intercept","coef":reg.intercept_})],axis=0)
df_reg['abs_coef']=df_reg['coef'].abs()
df_reg=df_reg.sort_values(by="abs_coef",ascending=False)
df_reg=df_reg.drop('abs_coef',axis=1)
df_reg


,features,coef
1,reason1,1.412572
3,reason3,0.762560
4,reason4,-0.590086
13,Children,0.447366
7,Transportation Expense,0.284117
9,Age,-0.194889
11,Body Mass Index,0.170678
0,intercept,0.165358
10,Daily Work Load Average,-0.143800
6,week_day,-0.130384


II.USING NEURAL NETWORK

In [96]:
import tensorflow


In [109]:
input_size=14
hidden_size=300
output_size=1


model=tensorflow.keras.Sequential([
    tensorflow.keras.layers.Dense(hidden_size,activation='tanh'),
    tensorflow.keras.layers.Dense(hidden_size,activation='tanh'),
    
    
    tensorflow.keras.layers.Dense(output_size,activation='sigmoid')])
optimizer=tensorflow.keras.optimizers.Adam()
model.compile(loss='binary_crossentropy',optimizer=optimizer,metrics='accuracy')


In [110]:

batch_size = 100

max_epochs = 100

# setting  an early stopping mechanism
# let's set patience=2, to be a bit tolerant against random validation loss increases
early_stopping = tensorflow.keras.callbacks.EarlyStopping(patience=2)

# fit the model

model.fit(x_train, # train inputs
          y_train, # train targets
          batch_size=batch_size, # batch size
          epochs=max_epochs, # epochs that we will train for (assuming early stopping doesn't kick in)
          callbacks=[early_stopping], # early stopping 
          validation_data=(x_val, y_val), # validation data
          #verbose = 1 # making sure we get enough information about the training process
          )  

Epoch 1/100
6/6 [==============================] - 3s 141ms/step - loss: 0.9349 - accuracy: 0.5103 - val_loss: 0.6984 - val_accuracy: 0.5333
Epoch 2/100
6/6 [==============================] - 0s 20ms/step - loss: 0.7406 - accuracy: 0.5421 - val_loss: 0.7107 - val_accuracy: 0.4833
Epoch 3/100
6/6 [==============================] - 0s 17ms/step - loss: 0.7060 - accuracy: 0.4953 - val_loss: 0.6874 - val_accuracy: 0.5333
Epoch 4/100
6/6 [==============================] - 0s 20ms/step - loss: 0.7030 - accuracy: 0.4598 - val_loss: 0.6826 - val_accuracy: 0.5333
Epoch 5/100
6/6 [==============================] - 0s 18ms/step - loss: 0.6804 - accuracy: 0.5607 - val_loss: 0.6793 - val_accuracy: 0.5333
Epoch 6/100
6/6 [==============================] - 0s 18ms/step - loss: 0.6754 - accuracy: 0.5645 - val_loss: 0.6716 - val_accuracy: 0.5500
Epoch 7/100
6/6 [==============================] - 0s 19ms/step - loss: 0.6815 - accuracy: 0.5121 - val_loss: 0.6877 - val_accuracy: 0.5333
Epoch 8/100
6/6 [==

In [111]:
loss,accuracy=model.evaluate(x_test,y_test)
print("accuracy =",accuracy)

4/4 [==============================] - 0s 7ms/step - loss: 0.5648 - accuracy: 0.6762
accuracy = 0.6761904954910278


III.USING RANDOM FOREST CLASSIFIER

In [112]:
from sklearn.ensemble import RandomForestClassifier


In [113]:
clf=RandomForestClassifier(n_estimators=100,random_state=42)
model=clf.fit(x_train,y_train)

In [114]:
outputs=model.predict(x_test)
#print(outputs)
accuracy_matrix=outputs==y_test
print(np.array([int(value) for value in accuracy_matrix]))

accuracy=sum(accuracy_matrix)/len(accuracy_matrix)
print(accuracy)

[0 1 1 0 1 0 1 0 1 0 1 1 1 1 1 1 1 1 0 1 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 0
 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 0 1 1 1 1 1 1 1 0 1 0 1 1 1 1 1 1 1 1 1 0 0
 1 0 0 1 1 0 0 0 1 1 1 1 1 1 1 1 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1]
0.7904761904761904
